# SIR-Model in Dynamic Dashboard

This notebook provides the implementation of the SIR model in a dynamic dashboard for delivery 4 of the course "Enterprise Data Science". Again, data from OurWorldInData is used.

Note to the function `fit_odeint`:
- Instead of returning the currently infected people, the **total cases is returned** by subtracting the susceptible people from the overall population N0. Only with that, the `ydata` containing the total number of cases summed up is fitted correctly. Otherwise, if only I (the **currently** infected number) is returned, the call `optimize.curve_fit(fit_odeint, t, ydata)` would try to fit I from the SIR model to the overall summed total cases which is not correct from a practical point of view.

# 1. Fetch New Data from OurWorldInData

In [1]:
# %load "../src/data/get_data.py"
import pandas as pd

def get_data():
    """ Get current data from Our World in Data """
    url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
    df_owid = pd.read_csv(url, sep=',')
    df_owid.to_csv("../data/raw/covid_full_data.csv", sep=";")


if __name__ == '__main__':
    get_data()

# 2. Define SIR Model and Global Variables

In [2]:
# %load "../src/models/sir_model.py"
from scipy import integrate

N0=0
S0=0
I0=0
R0=0
t=0

def SIR_model_t(SIR,t,beta,gamma):
    global N0
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration.
    Modification made: fit the summed cases N0-S instead of the active cases I as the provided/compared data is also the summed up total case number.
    '''
    global S0, I0, R0, t
    return N0 - integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,0]    # return total summed cases (=N0-S)

# 3. Visualization

In [3]:
# %load "../src/visualization/visualize_sir.py"
import pandas as pd
import numpy as np

import plotly.graph_objects as go

import dash
from dash import dcc as doc
from dash.dependencies import Input, Output
from dash import html
import dash_daq as daq

from scipy import optimize

fig_sir = go.Figure()
app = dash.Dash()

df_covid = pd.read_csv("../data/raw/covid_full_data.csv", sep=";")
countries = df_covid['location'].unique()

app.layout = html.Div([
    html.H1('Dynamic Covid-19 Dashboard'),
    daq.BooleanSwitch(id='loglin_switch', on=False, label="Logarithmic scale", labelPosition="top"),
    html.Label('Select the countries to display:'),
    doc.Dropdown(
        id = 'country_drop_down',
        options=[{'label': country, 'value': country} for country in countries],
        value=['Germany'],        # which are pre-selected
        multi=True
    ),
    doc.Graph(figure=fig_sir,id='main_window_sir')
])

@app.callback(
    Output('main_window_sir', 'figure'),
    [Input('country_drop_down', 'value'), Input('loglin_switch', 'on')])
def update_figure(countries_to_show, switch_state):
    global N0, S0, I0, R0, t
    traces = []
    for country in countries_to_show:
        total_cases_country = df_covid.total_cases[df_covid.location==country]
        ydata = np.array(total_cases_country.iloc[50:180])
        t=np.arange(len(ydata))
        
        # use population of country from dataframe
        N0 = df_covid.population[df_covid.location==country].iloc[60];
        I0=ydata[0]          # Initial value of infected people
        S0=N0-I0             # Initial value for sususceptible people
        R0=0                 # Initial value for recovered people
        # the resulting curve has to be fitted
        # free parameters are here beta and gamma
        
        popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)       # popt contains fitted beta and gamma
        perr = np.sqrt(np.diag(pcov))
        print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
        print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
        # get the final fitted curve
        fitted = fit_odeint(t, *popt)
        print(N0, I0, S0)
        
        traces.append(dict(x=t,
                             y=ydata,
                             name=f"Total (Summed) Cases {country}",
                             opacity=0.9,
                             line_width=2,
                             marker_size=4,
                             mode='markers'
                          )
                     )
        traces.append(dict(x=t,
                             y=fitted,
                             name=f"Total Cases according to SIR-model {country} (=N0-S)",
                             line_width=2,
                             marker_size=4,
                             mode='lines'
                          )
                     )
        
    return {
        'data': traces,
        'layout': dict(width=1280,
                        height=720,
                        title="Fit of SIR model for selected countries",
                        xaxis={'tickangle':-45,
                              'nticks':20,
                              'tickfont':dict(size=14,color='#7f7f7f'),
                               'title':'Days',
                              },
                        yaxis={
                            'type': ('log' if switch_state else 'linear'),
                            'range':('[0.1,100]' if switch_state else '[0,100000000]'),
                            'title':'Population Infected'
                        })
    }

app.run(debug=True, use_reloader=False, port=8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
standard deviation errors :  [0.00995396 0.00997698]  start infect: 7156.0
Optimal parameters: beta = 0.8282730381781627  and gamma =  0.8574025747465149
83408554.0 7156.0 83401398.0
standard deviation errors :  [0.00995396 0.00997698]  start infect: 7156.0
Optimal parameters: beta = 0.8282730381781627  and gamma =  0.8574025747465149
83408554.0 7156.0 83401398.0
standard deviation errors :  [0.00995396 0.00997698]  start infect: 7156.0
Optimal parameters: beta = 0.8282730381781627  and gamma =  0.8574025747465149
83408554.0 7156.0 83401398.0
standard deviation errors :  [0.00995396 0.00997698]  start infect: 7156.0
Optimal parameters: beta = 0.8282730381781627  and gamma =  0.8574025747465149
83408554.0 7156.0 83401398.0
standard deviation errors :  [0.00995396 0.00997698]  start infect: 7156.0
Optimal parameter